In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

Suppose that in a certain defined benefit pension plan
    - (a) Employees work for 45 years earning wages that increase at a real rate of 2%.
    - (b) They retire with a pension equal to 70% of their final salary. This pension increases at the rate of in ation minus 1%.
    - (c) The pension is received for 18 years.
    - (d) The pension fund’s income is invested in bonds which earn the in ation rate plus 1.5%.


Estimate the percentage of an employee’s salary that must be contributed to the pension plan if it is to remain solvent. (Hint: Do all calculations in real rather than nominal dollars.)

# Q4

In [2]:
def breakeven_contribution(c, base=100, g=0.02, r=0.015, incre=-0.01, T1=45, T2=18, return_table=False):
    
    work = pd.DataFrame({'Bal': np.zeros(T1)}, index=range(1, T1+1))
    retire = pd.DataFrame({'Bal': np.zeros(T2)}, index=range(1, T2+1))
    
    idx = np.array(range(T1)) + 1

    work['Wage'] = np.ones(T1) * np.power(1+g,idx-1) * base
    work['Contribution'] = work['Wage']*c

    work.loc[1,'Bal']=work.loc[1,'Contribution']
    for i in idx[1:]:
        work.loc[i,'Bal']=work.loc[i-1,'Bal']*(1+r)+work.loc[i,'Contribution']
        
    idx = np.array(range(T2)) + 1
    pensions = np.ones(T2) * np.power(1+incre,idx-1) * work['Wage'].values[-1] * 0.7
    retire['pansion'] = pensions
    
    retire.loc[1,'Bal']=work.loc[T1,'Bal'] * (1+r) - retire.loc[1, 'pansion']

    for i in idx[1:]:
        retire.loc[i,'Bal']=retire.loc[i-1,'Bal']*(1+r) - retire.loc[i,'pansion']
    
    return retire['Bal'].values[-1], (work, retire) if return_table else retire['Bal'].values[-1]

In [3]:
from scipy import optimize

c = optimize.fsolve(breakeven_contribution,0)
_, df = breakeven_contribution(c, return_table=True)

print c

[ 0.25022212]


/Users/jenchieh/anaconda2/lib/python2.7/site-packages/scipy/optimize/minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [4]:
df[0].round(2)

,Bal,Wage,Contribution
1,25.02,100.00,25.02
2,50.92,102.00,25.52
3,77.72,104.04,26.03
4,105.44,106.12,26.55
5,134.10,108.24,27.08
6,163.74,110.41,27.63
7,194.38,112.62,28.18
8,226.03,114.87,28.74
9,258.74,117.17,29.32
10,292.53,119.51,29.90


In [5]:
df[1].round(2)

,Bal,pansion
1,2289.36,167.30
2,2158.07,165.63
3,2026.46,163.97
4,1894.52,162.33
5,1762.23,160.71
6,1629.56,159.10
7,1496.49,157.51
8,1363.00,155.94
9,1229.07,154.38
10,1094.67,152.83


## Q5

In [6]:
salary = 800
N = 500
T1, T2 = 45, 18
Z = np.random.normal(.015, .06, (N, T1+T2))
Z = np.r_[Z, -Z]

In [7]:
g = 0.02
incre = -0.01

r = 0.015
v = 0.06
bond_return = lambda z: np.exp(r - v*v*0.5 + v*z)

contri = np.ones(T1) * np.power(1+g,range(T1)) * salary * c
pansion = np.ones(T2) * np.power(1+incre,range(T2)) * salary * ((1+g)**(T1-1)) * 0.7
cf = np.r_[contri, -pansion]
bal = [np.ones(N*2) * cf[0]]

for i in range(1, T1+T2):
    bal.append(bal[-1]*bond_return(Z[:,i-1])+cf[i])

#matrix: (2xN)x(T1+T2)
bal = np.array(bal).T 

In [8]:
ante_fr = []
post_fr = []

for i in range(T1,T1+T2):

    ante_L = -np.sum(np.minimum(cf[i:], 0) / np.power(1+r, range(i, T1+T2)))
    ante_A = bal[:, i] - cf[i]
    
    post_L = -np.sum(np.minimum(cf[i+1:], 0) / np.power(1+r, range(i+1, T1+T2)))
    post_A = bal[:, i]
    
    ante_fr.append((ante_A / ante_L))
    if (i !=T1+T2-1):
        post_fr.append((post_A / post_L))


ante_fr = np.array(ante_fr)
post_fr = np.array(post_fr)

ante_events = (ante_fr < 0.8)*1
post_events = (post_fr < 0.8)*1

In [9]:
# ante basis

print ante_events.any(axis=0).mean()
pd.DataFrame({'E(F.R.)': ante_fr.mean(axis=1), 'P(FR < .8)': ante_events.mean(axis=1)}, 
             index=range(T1+1,T1+T2+1))

0.677


,E(F.R.),P(FR < .8)
46,1.883541,0.000
47,1.903381,0.000
48,1.922484,0.000
49,1.940630,0.000
50,1.957558,0.000
51,1.972869,0.000
52,1.986078,0.000
53,1.996530,0.000
54,2.003167,0.000
55,2.004687,0.000


In [10]:
# post basis
print post_events.any(axis=0).mean()
pd.DataFrame({'E(F.R.)': post_fr.mean(axis=1), 'P(FR < .8)': post_events.mean(axis=1)}, 
             index=range(T1+1,T1+T2))

0.677


,E(F.R.),P(FR < .8)
46,1.878376,0.000
47,1.897230,0.000
48,1.915137,0.000
49,1.931817,0.000
50,1.946927,0.000
51,1.959944,0.000
52,1.970212,0.000
53,1.976813,0.000
54,1.978267,0.000
55,1.972611,0.000
